In [ ]:
import numpy as np
import mercantile
import matplotlib.pyplot as plt
from elevation import getElevationMatrix, rasterToImage, getRasterRGB

# from local_config import MAPBOX_TOKEN
import math

MAPBOX_TOKEN = "pk.eyJ1IjoiY3Jpc3BpYW5tIiwiYSI6ImNsMG1oazJhejE0YzAzZHVvd2Z1Zjlhb2YifQ.cv0zlPYY6WnoKM9YLD1lMQ"

"""tile_coords = mercantile.tile(lng=-95.9326171875, lat=41.26129149391987, zoom=12)
print(tile_coords)
elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)
padded_mat = np.pad(elevation_mat, [(1, 1), (1, 1)], mode='constant', constant_values=np.Inf)
print(padded_mat)
# Get latitude and longitude at upper-left of tile
upper_left = mercantile.ul(tile_coords)
print("upperleft:", upper_left)"""


def get_tile(lat, lng, zoom_level):
    tile_coords = mercantile.tile(lng=lng, lat=lat, zoom=zoom_level)
    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )
    # print(padded_mat, flush=True)
    # Get latitude and longitude at upper-left of tile
    upper_left = mercantile.ul(tile_coords)
    djikstra(
        padded_mat,
        startNode=(1, 1),
        targetNode=(248, 250),
        zoomlevel=zoom_level,
        latitude=lat,
        elevation_multiplier=10,
        show_plot=True,
    )


def construct_lng_lat_matrix(ul, zoomlevel):

    matrix = np.zeros([256, 256], list)

    for i in range(256):
        for j in range(256):
            lng_lat = coord_to_lng_lat(ul, coord=(i, j), zoomlevel=zoomlevel)
            matrix[i, j] = [lng_lat[0], lng_lat[1]]

    return matrix


def construct_lng_lat_matrix2(tile):

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = round(bbox.west, 6)
    ul_lat = round(bbox.north, 6)
    # Unpack lower-right lng lat
    lr_lng = round(bbox.east, 6)
    lr_lat = round(bbox.south, 6)

    # delta lng & lat from upper-left to lower-right
    delta_lng = round(abs(1000000*bbox.east - 1000000*bbox.west), 6)
    delta_lat = round(abs(1000000*bbox.north - 1000000*bbox.south), 6)

    matrix = np.zeros([256, 256], list)

    for i in range(256):
        for j in range(256):
            cell_lng = round(1000000*ul_lng + (j*delta_lng)/256, 6)
            cell_lat = round(1000000*ul_lat - (i*delta_lat)/256, 6)
            matrix[i,j] = [round(cell_lng/1000000, 6), round(cell_lat/1000000, 6)]

    return matrix


def coord_to_lng_lat2(ul, coord, zoomlevel):
    """Converts x,y matrix coordinate into longitude and latitude coordinates"""
    # Unpack upper-left of tile longitude and latitude
    ul_lat = ul.lat
    ul_lng = ul.lng

    # Calculate distance between each pixel
    latitude_radians = ul_lat * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**zoomlevel))

    # Radius of Earth in metres
    R = 6378137

    # Change in distance (delta pixels * resolution in metres)
    dn = coord[0] * resolution
    de = coord[1] * resolution

    dLat = dn / R
    dLon = de / (R * math.cos(math.pi * ul_lat / 180))

    latO = ul_lat + dLat * 180 / math.pi
    lonO = ul_lng + dLon * 180 / math.pi

    return [lonO, latO]


def coord_to_lng_lat(coord, tile):

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = bbox.west
    ul_lat = bbox.north
    # Unpack lower-right lng lat
    lr_lng = bbox.east
    lr_lat = bbox.south

    # delta lng & lat from upper-left to lower-right
    total_delta_lng = abs(abs(lr_lng) - abs(ul_lng))
    total_delta_lat = abs(abs(lr_lat) - abs(ul_lat))

    lng = ul_lng + (coord[0]/255)*total_delta_lng
    lat = ul_lat - (coord[1]/255)*total_delta_lat

    return (lng, lat)


def lng_lat_to_coord(lng_lat, tile):

    target_lng = lng_lat[0]
    target_lat = lng_lat[1]

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = bbox.west
    ul_lat = bbox.north
    # Unpack lower-right lng lat
    lr_lng = bbox.east
    lr_lat = bbox.south

    # delta lng & lat from upper-left to lower-right
    total_delta_lng = abs(abs(lr_lng) - abs(ul_lng))
    total_delta_lat = abs(abs(lr_lat) - abs(ul_lat))
    target_delta_lng = abs(abs(target_lng) - abs(ul_lng))
    target_delta_lat = abs(abs(target_lat) - abs(ul_lat))

    x = (target_delta_lng / total_delta_lng)
    y = (target_delta_lat / total_delta_lat)

    x = math.floor(x*255)
    y = math.floor(y*255)
    return (x, y)


def djikstra(matrix, startNode, targetNode, resolution, elevation_multiplier=4):

    neighbourDiffs = [[0, 1], [0, -1], [-1, 0], [1, 0]]
    visitedNodes = {
        startNode: 0
    }  # Dictionary of nodes and their shortest discovered cumulative distance
    frontierNodes = dict()
    parentDict = dict()

    currentNode = startNode
    currentDist = 0
    # print("startNode:", startNode)
    # print("targetNode:", targetNode)
    while True:

        neighbourNodes = set(
            tuple(np.array(currentNode) + np.array(diff)) for diff in neighbourDiffs
        )

        for node in neighbourNodes:
            if node not in visitedNodes.keys():
                # Generate weighting for traversing to neighbouring node
                neighbourDist = (
                    currentDist
                    + resolution
                    + elevation_multiplier * abs((float(matrix[node]) - float(matrix[currentNode])))
                )

                # Update frontier if newly-discovered distance is smaller than existing frontier distance
                try:
                    if neighbourDist < frontierNodes[node]:
                        frontierNodes[node] = neighbourDist
                        # Update parent node for shortest path
                        parentDict[node] = currentNode
                except KeyError:
                    frontierNodes[node] = neighbourDist
                    parentDict[node] = currentNode

        # Search frontier nodes for smallest distance
        smallestFrontierNode = min(frontierNodes, key=frontierNodes.get)

        # Change current node to smallest frontier node
        currentNode = smallestFrontierNode
        currentDist = frontierNodes[currentNode]

        # Remove new current node from frontier
        frontierNodes.pop(currentNode, None)

        # Add new current node to visited nodes
        visitedNodes[currentNode] = currentDist
        if targetNode in visitedNodes.keys():
            # print("DONE")
            break

    # print(len(visitedNodes))

    # Backtracking to get path of nodes
    currentNode = targetNode
    nodePath = [currentNode]
    while currentNode != startNode:
        currentNode = parentDict[currentNode]
        nodePath.append(currentNode)
    # print(nodePath)

    return nodePath


def get_min_path(
    start_lng_lat, end_lng_lat, zoom, elevation_multiplier=5, show_img=False
):

    # Get direction of end location relative to start
    x_delta = end_lng_lat[0] - start_lng_lat[0]
    y_delta = end_lng_lat[1] - start_lng_lat[1]

    if (x_delta < 0) and (y_delta > 0):
        tile_lng_lat = end_lng_lat
        #startNode = (256, 256)

    elif (x_delta < 0) and (y_delta < 0):
        tile_lng_lat = (end_lng_lat[0], start_lng_lat[1])
        #startNode = (1, 256)

    elif (x_delta > 0) and (y_delta < 0):
        tile_lng_lat = start_lng_lat
        #startNode = (1, 1)

    elif (x_delta > 0) and (y_delta > 0):
        tile_lng_lat = (start_lng_lat[0], end_lng_lat[1])
        #startNode = (256, 1)

    # Get mercantile tile x,y,z from lng, lat, zoom
    tile_coords = mercantile.tile(lng=tile_lng_lat[0], lat=tile_lng_lat[1], zoom=zoom)

    # Get elevation matrix
    elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)
    # Pad matrix with infinities to represent boundaries
    padded_mat = np.pad(elevation_mat, [(1, 1), (1, 1)], mode='constant', constant_values=np.Inf)

    # resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
    latitude_radians = tile_lng_lat[1] * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2 ** zoom))

    startNode = lng_lat_to_coord(start_lng_lat, tile_coords)
    targetNode = lng_lat_to_coord(end_lng_lat, tile_coords)

    # Generate the shortest path as a sequence of lng, lat tuples
    node_path = djikstra(
        padded_mat,
        startNode=(startNode[0], startNode[1]),
        targetNode=(targetNode[0], targetNode[1]),
        resolution=resolution,
        elevation_multiplier=elevation_multiplier,
    )

    # Gets path as series of longitude and latitude coordinates
    lnglatPath = [coord_to_lng_lat(coord, tile_coords) for coord in node_path]

    if show_img:
        plt.imshow(elevation_mat, interpolation="nearest")
        xs = [x[0] for x in node_path]
        ys = [x[1] for x in node_path]
        plt.plot(xs, ys, "r-")
        plt.show()

    return lnglatPath


def get_min_path_from_bbox(bbox, elevation_multiplier=10, show_img=False):
    if bbox[0] > bbox[2]:  # coord 1 more east
        if bbox[1] > bbox[3]:  # coord 1 more north
            bbox = [bbox[2], bbox[1], bbox[0], bbox[3]]
        else:  # coord 1 more south
            bbox = [bbox[2], bbox[3], bbox[0], bbox[1]]
    else:  # coord 1 more west
        if bbox[1] > bbox[3]:  # coord 1 more north
            pass
        else:  # coord 1 more south
            bbox = [bbox[0], bbox[3], bbox[2], bbox[1]]

    tile_coords = mercantile.bounding_tile(bbox[0], bbox[1], bbox[2], bbox[3])

    upper_left = mercantile.ul(tile_coords)

    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    lng_lat_mat = construct_lng_lat_matrix(upper_left, tile_coords.z)

    # Pad matrix with infinities to represent boundaries
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )

    # resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
    latitude_radians = bbox[1] * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**tile_coords.z))

    startNode = lng_lat_to_coord(lng_lat_mat, [bbox[0], bbox[1]])
    targetNode = lng_lat_to_coord(lng_lat_mat, [bbox[2], bbox[3]])

    # Generate the shortest path as a sequence of lng, lat tuples
    node_path = djikstra(
        padded_mat,
        startNode=(startNode[0] + 1, startNode[1] + 1),
        targetNode=(targetNode[0] + 1, targetNode[1] + 1),
        resolution=resolution,
        elevation_multiplier=elevation_multiplier,
    )

    # Gets path as series of longitude and latitude coordinates
    lnglatPath = [
        coord_to_lng_lat(upper_left, coord, tile_coords.z) for coord in node_path
    ]

    if show_img:
        plt.imshow(elevation_mat, interpolation="nearest")
        xs = [x[0] for x in node_path]
        ys = [x[1] for x in node_path]
        plt.plot(xs, ys, "r-")
        plt.show()

    return lnglatPath


In [1]:
import re, math
from flask import json
import numpy as np
# from new_pathfinding import construct_lng_lat_matrix, get_min_path
import mercantile
from elevation import getElevationMatrix, rasterToImage, getRasterRGB

MAPBOX_TOKEN = "pk.eyJ1IjoiY3Jpc3BpYW5tIiwiYSI6ImNsMG1oazJhejE0YzAzZHVvd2Z1Zjlhb2YifQ.cv0zlPYY6WnoKM9YLD1lMQ"

In [2]:
from pathfinding import get_min_path
from new_pathfinding import new_get_min_path

def convertToJson(minPath):
    geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {"type": "LineString", "coordinates": minPath},
            }
        ],
    }
    output = open("old_minPath.geojson", "w")
    json.dump(geojson, output)
    return geojson

def new_convertToJson(minPath):
    geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {"type": "LineString", "coordinates": minPath},
            }
        ],
    }
    output = open("new_minPath.geojson", "w")
    json.dump(geojson, output)
    return geojson

In [10]:
def get_tile(lat, lng, zoom_level):
    tile_coords = mercantile.tile(lng=lng, lat=lat, zoom=zoom_level)
    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )
    # print(padded_mat, flush=True)
    # Get latitude and longitude at upper-left of tile
    upper_left = mercantile.ul(tile_coords)
    djikstra(
        padded_mat,
        startNode=(1, 1),
        targetNode=(248, 250),
        zoomlevel=zoom_level,
        latitude=lat,
        elevation_multiplier=10,
        show_plot=True,
    )


def construct_lng_lat_matrix(ul, zoomlevel):

    matrix = np.zeros([256, 256], list)

    for i in range(256):
        for j in range(256):
            lng_lat = coord_to_lng_lat(ul, coord=(i, j), zoomlevel=zoomlevel)
            matrix[i, j] = [lng_lat[0], lng_lat[1]]

    return matrix


def construct_lng_lat_matrix2(tile):

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = round(bbox.west, 6)
    ul_lat = round(bbox.north, 6)
    # Unpack lower-right lng lat
    lr_lng = round(bbox.east, 6)
    lr_lat = round(bbox.south, 6)

    # delta lng & lat from upper-left to lower-right
    delta_lng = round(abs(1000000 * bbox.east - 1000000 * bbox.west), 6)
    delta_lat = round(abs(1000000 * bbox.north - 1000000 * bbox.south), 6)

    matrix = np.zeros([256, 256], list)

    for i in range(256):
        for j in range(256):
            cell_lng = round(1000000 * ul_lng + (j * delta_lng) / 256, 6)
            cell_lat = round(1000000 * ul_lat - (i * delta_lat) / 256, 6)
            matrix[i, j] = [round(cell_lng / 1000000, 6), round(cell_lat / 1000000, 6)]

    return matrix


def coord_to_lng_lat2(ul, coord, zoomlevel):
    """Converts x,y matrix coordinate into longitude and latitude coordinates"""
    # Unpack upper-left of tile longitude and latitude
    ul_lat = ul.lat
    ul_lng = ul.lng

    # Calculate distance between each pixel
    latitude_radians = ul_lat * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**zoomlevel))

    # Radius of Earth in metres
    R = 6378137

    # Change in distance (delta pixels * resolution in metres)
    dn = coord[0] * resolution
    de = coord[1] * resolution

    dLat = dn / R
    dLon = de / (R * math.cos(math.pi * ul_lat / 180))

    latO = ul_lat + dLat * 180 / math.pi
    lonO = ul_lng + dLon * 180 / math.pi

    return [lonO, latO]


def coord_to_lng_lat(coord, tile):

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = bbox.west
    ul_lat = bbox.north
    # Unpack lower-right lng lat
    lr_lng = bbox.east
    lr_lat = bbox.south

    # delta lng & lat from upper-left to lower-right
    total_delta_lng = abs(abs(lr_lng) - abs(ul_lng))
    total_delta_lat = abs(abs(lr_lat) - abs(ul_lat))

    lng = ul_lng + (coord[0] / 255) * total_delta_lng
    lat = ul_lat - (coord[1] / 255) * total_delta_lat

    return (lng, lat)


def lng_lat_to_coord(lng_lat, tile):

    target_lng = lng_lat[0]
    target_lat = lng_lat[1]

    bbox = mercantile.bounds(tile)
    # Unpack upper-left lng lat
    ul_lng = bbox.west
    ul_lat = bbox.north
    # Unpack lower-right lng lat
    lr_lng = bbox.east
    lr_lat = bbox.south

    # delta lng & lat from upper-left to lower-right
    total_delta_lng = abs(abs(lr_lng) - abs(ul_lng))
    total_delta_lat = abs(abs(lr_lat) - abs(ul_lat))
    target_delta_lng = abs(abs(target_lng) - abs(ul_lng))
    target_delta_lat = abs(abs(target_lat) - abs(ul_lat))

    x = target_delta_lng / total_delta_lng
    y = target_delta_lat / total_delta_lat

    x = math.floor(x * 255)
    y = math.floor(y * 255)
    return (x, y)


def get_min_path(
    start_lng_lat, end_lng_lat, zoom, elevation_multiplier=5, show_img=False
):

    # Get direction of end location relative to start
    x_delta = end_lng_lat[0] - start_lng_lat[0]
    y_delta = end_lng_lat[1] - start_lng_lat[1]

    if (x_delta < 0) and (y_delta > 0):
        tile_lng_lat = end_lng_lat
        # startNode = (256, 256)

    elif (x_delta < 0) and (y_delta < 0):
        tile_lng_lat = (end_lng_lat[0], start_lng_lat[1])
        # startNode = (1, 256)

    elif (x_delta > 0) and (y_delta < 0):
        tile_lng_lat = start_lng_lat
        # startNode = (1, 1)

    elif (x_delta > 0) and (y_delta > 0):
        tile_lng_lat = (start_lng_lat[0], end_lng_lat[1])
        # startNode = (256, 1)

    # Get mercantile tile x,y,z from lng, lat, zoom
    tile_coords = mercantile.tile(lng=tile_lng_lat[0], lat=tile_lng_lat[1], zoom=zoom)

    # Get elevation matrix
    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    # Pad matrix with infinities to represent boundaries
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )

    # resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
    latitude_radians = tile_lng_lat[1] * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**zoom))

    startNode = lng_lat_to_coord(start_lng_lat, tile_coords)
    targetNode = lng_lat_to_coord(end_lng_lat, tile_coords)

    # Generate the shortest path as a sequence of lng, lat tuples
    node_path = djikstra(
        padded_mat,
        startNode=(startNode[0], startNode[1]),
        targetNode=(targetNode[0], targetNode[1]),
        resolution=resolution,
        elevation_multiplier=elevation_multiplier,
    )

    # Gets path as series of longitude and latitude coordinates
    lnglatPath = [coord_to_lng_lat(coord, tile_coords) for coord in node_path]

    if show_img:
        plt.imshow(elevation_mat, interpolation="nearest")
        xs = [x[0] for x in node_path]
        ys = [x[1] for x in node_path]
        plt.plot(xs, ys, "r-")
        plt.show()

    return lnglatPath


def get_min_path_from_bbox(bbox, elevation_multiplier=10, show_img=False):
    if bbox[0] > bbox[2]:  # coord 1 more east
        if bbox[1] > bbox[3]:  # coord 1 more north
            bbox = [bbox[2], bbox[1], bbox[0], bbox[3]]
        else:  # coord 1 more south
            bbox = [bbox[2], bbox[3], bbox[0], bbox[1]]
    else:  # coord 1 more west
        if bbox[1] > bbox[3]:  # coord 1 more north
            pass
        else:  # coord 1 more south
            bbox = [bbox[0], bbox[3], bbox[2], bbox[1]]

    tile_coords = mercantile.bounding_tile(bbox[0], bbox[1], bbox[2], bbox[3])

    upper_left = mercantile.ul(tile_coords)

    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    lng_lat_mat = construct_lng_lat_matrix(upper_left, tile_coords.z)

    # Pad matrix with infinities to represent boundaries
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )

    # resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
    latitude_radians = bbox[1] * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**tile_coords.z))

    startNode = lng_lat_to_coord(lng_lat_mat, [bbox[0], bbox[1]])
    targetNode = lng_lat_to_coord(lng_lat_mat, [bbox[2], bbox[3]])

    # Generate the shortest path as a sequence of lng, lat tuples
    node_path = djikstra(
        padded_mat,
        startNode=(startNode[0] + 1, startNode[1] + 1),
        targetNode=(targetNode[0] + 1, targetNode[1] + 1),
        resolution=resolution,
        elevation_multiplier=elevation_multiplier,
    )

    # Gets path as series of longitude and latitude coordinates
    lnglatPath = [
        coord_to_lng_lat(upper_left, coord, tile_coords.z) for coord in node_path
    ]

    if show_img:
        plt.imshow(elevation_mat, interpolation="nearest")
        xs = [x[0] for x in node_path]
        ys = [x[1] for x in node_path]
        plt.plot(xs, ys, "r-")
        plt.show()

    return lnglatPath



def djikstra(matrix, startNode, targetNode, resolution, elevation_multiplier=4):

    neighbourDiffs = [[0, 1], [0, -1], [-1, 0], [1, 0]]
    visitedNodes = {
        startNode: 0
    }  # Dictionary of nodes and their shortest discovered cumulative distance
    frontierNodes = dict()
    parentDict = dict()

    currentNode = startNode
    currentDist = 0
    # print("startNode:", startNode)
    # print("targetNode:", targetNode)
    while True:

        neighbourNodes = set(
            tuple(np.array(currentNode) + np.array(diff)) for diff in neighbourDiffs
        )

        for node in neighbourNodes:
            if node not in visitedNodes.keys():
                # Generate weighting for traversing to neighbouring node
                neighbourDist = (
                    currentDist
                    + resolution
                    + elevation_multiplier * abs((matrix[node] - matrix[currentNode]))
                )

                # Update frontier if newly-discovered distance is smaller than existing frontier distance
                try:
                    if neighbourDist < frontierNodes[node]:
                        frontierNodes[node] = neighbourDist
                        # Update parent node for shortest path
                        parentDict[node] = currentNode
                except KeyError:
                    frontierNodes[node] = neighbourDist
                    parentDict[node] = currentNode

        # Search frontier nodes for smallest distance
        smallestFrontierNode = min(frontierNodes, key=frontierNodes.get)

        # Change current node to smallest frontier node
        currentNode = smallestFrontierNode
        currentDist = frontierNodes[currentNode]

        # Remove new current node from frontier
        frontierNodes.pop(currentNode, None)

        # Add new current node to visited nodes
        visitedNodes[currentNode] = currentDist
        if targetNode in visitedNodes.keys():
            # print("DONE")
            break

    # print(len(visitedNodes))

    # Backtracking to get path of nodes
    currentNode = targetNode
    nodePath = [currentNode]
    while currentNode != startNode:
        currentNode = parentDict[currentNode]
        nodePath.append(currentNode)
    # print(nodePath)

    return nodePath

In [ ]:
# crisp_start_click = '{"x":1216,"y":525}<br>{"lng":-2.619232,"lat":51.449311}'
# crisp_end_click = '{"x":1216,"y":525}<br>{"lng":-2.614962,"lat":51.448901}'

# crisp_start_latlonDict = json.loads(re.findall("\{.*?\}", start_click)[1])
# crisp_end_latlonDict = json.loads(re.findall("\{.*?\}", end_click)[1])

# crisp_zoom_level = 14.1561

# crisp_startPoint = [start_latlonDict["lng"], start_latlonDict["lat"]]
# crisp_endPoint = [end_latlonDict["lng"], end_latlonDict["lat"]]

# min_path = get_min_path(
#     crisp_startPoint, crisp_endPoint, math.ceil(float(zoom_level))
# )

In [49]:
start_click = '{"x":1216,"y":525}<br>{"lng":-2.619232,"lat":51.449311}'
end_click = '{"x":1216,"y":525}<br>{"lng":-2.614962,"lat":51.448901}'

startPoint =  [-4.022497799944472, 53.01103695333193]
endPoint =  [-4.074585773503941, 53.07309152521492]

# startPoint =  [-4.740760655580857, 57.29077722371997]
# endPoint = [-4.711469073425405, 57.23137895009063]

start_latlonDict = json.loads(re.findall("\{.*?\}", start_click)[1])
end_latlonDict = json.loads(re.findall("\{.*?\}", end_click)[1])



# startPoint = [start_latlonDict["lng"], start_latlonDict["lat"]]
# endPoint = [end_latlonDict["lng"], end_latlonDict["lat"]]

print("startPoint: ", startPoint)
print("endPoint: ", endPoint)


zoom_level = math.ceil(float(self.zoom_level))
min_path = []
while len(min_path) == 0:
    try:
        min_path = new_get_min_path(startPoint, endPoint, zoom_level)
    except:
        print("Error at zoom level: ", zoom_level)
        zoom_level -= 1

# min_path = new_get_min_path(
#     startPoint, endPoint, zoom_level
# )

print(zoom_level)
print(min_path)
# crispy = new_convertToJson(min_path)

startPoint:  [-4.022497799944472, 53.01103695333193]
endPoint:  [-4.074585773503941, 53.07309152521492]
Error at zoom level:  16
Error at zoom level:  15
Error at zoom level:  14
Error at zoom level:  13
Error at zoom level:  12
Error at zoom level:  11
10
[[-4.022977941176471, 53.01175067985537], [-4.024356617647059, 53.01175067985537], [-4.025735294117647, 53.01175067985537], [-4.0271139705882355, 53.01175067985537], [-4.028492647058823, 53.01175067985537], [-4.029871323529412, 53.01175067985537], [-4.03125, 53.01175067985537], [-4.032628676470588, 53.01175067985537], [-4.034007352941177, 53.01175067985537], [-4.0353860294117645, 53.01175067985537], [-4.0353860294117645, 53.012580104307666], [-4.0353860294117645, 53.013409528759965], [-4.0353860294117645, 53.01423895321226], [-4.0353860294117645, 53.01506837766456], [-4.0353860294117645, 53.01589780211686], [-4.0353860294117645, 53.01672722656916], [-4.0353860294117645, 53.01755665102146], [-4.0353860294117645, 53.01838607547376], [-

In [42]:
zoom_level = 10
new_get_min_path(
        startPoint, endPoint, zoom_level
    )

[[-4.022977941176471, 53.01175067985537],
 [-4.024356617647059, 53.01175067985537],
 [-4.025735294117647, 53.01175067985537],
 [-4.0271139705882355, 53.01175067985537],
 [-4.028492647058823, 53.01175067985537],
 [-4.029871323529412, 53.01175067985537],
 [-4.03125, 53.01175067985537],
 [-4.032628676470588, 53.01175067985537],
 [-4.034007352941177, 53.01175067985537],
 [-4.0353860294117645, 53.01175067985537],
 [-4.0353860294117645, 53.012580104307666],
 [-4.0353860294117645, 53.013409528759965],
 [-4.0353860294117645, 53.01423895321226],
 [-4.0353860294117645, 53.01506837766456],
 [-4.0353860294117645, 53.01589780211686],
 [-4.0353860294117645, 53.01672722656916],
 [-4.0353860294117645, 53.01755665102146],
 [-4.0353860294117645, 53.01838607547376],
 [-4.0353860294117645, 53.019215499926055],
 [-4.0353860294117645, 53.02004492437836],
 [-4.0353860294117645, 53.02087434883066],
 [-4.0353860294117645, 53.02170377328296],
 [-4.0353860294117645, 53.02253319773526],
 [-4.0353860294117645, 53.

In [248]:
# # BAD
# startPoint = [location1["lat"], location1["lng"]]
# endPoint = [location2["lat"], location2["lng"]]

# route_to_place = get_min_path(startPoint, endPoint, 14)
# minpath = convertToJson(route_to_place)


# minpath["features"][0]["geometry"]["coordinates"][:] = map(
#     lambda l: list(reversed(l)),
#     minpath["features"][0]["geometry"]["coordinates"],
# )
# # minpath

In [249]:
start_lng_lat

[-2.619232, 51.449311]

In [250]:
end_lng_lat

[-2.614962, 51.448901]

In [251]:
# Get direction of end location relative to start
x_delta = end_lng_lat[0] - start_lng_lat[0]
x_delta

0.004269999999999996

In [252]:
y_delta = end_lng_lat[1] - start_lng_lat[1]
y_delta

-0.0004100000000022419

In [253]:
if (x_delta < 0) and (y_delta > 0):
    tile_lng_lat = end_lng_lat
    startNode = (256, 256)

elif (x_delta < 0) and (y_delta < 0):
    tile_lng_lat = (end_lng_lat[0], start_lng_lat[1])
    startNode = (1, 256)

elif (x_delta > 0) and (y_delta < 0):
    tile_lng_lat = start_lng_lat
    startNode = (1, 1)

elif (x_delta > 0) and (y_delta > 0):
    tile_lng_lat = (start_lng_lat[0], end_lng_lat[1])
    startNode = (256, 1)

print(tile_lng_lat)
print(startNode)

[-2.619232, 51.449311]
(1, 1)


In [254]:
# Get mercantile tile x,y,z from lng, lat, zoom
tile_coords = mercantile.tile(lng=tile_lng_lat[0], lat=tile_lng_lat[1], zoom=14)
print('tile_coords:', tile_coords)
upper_left = mercantile.ul(tile_coords)
print('\nupper_left: ', upper_left)

tile_coords: Tile(x=8072, y=5452, z=14)

upper_left:  LngLat(lng=-2.63671875, lat=51.45400691005982)


In [255]:
upper_left.lat

51.45400691005982

In [256]:
lng_lat_matrix = construct_lng_lat_matrix(upper_left, zoomlevel=14)
lng_lat_matrix

array([[list([-2.63671875, 51.45400691005982]),
        list([-2.6366329193136773, 51.45400691005982]),
        list([-2.636547088627354, 51.45400691005982]), ...,
        list([-2.615003586360315, 51.45400691005982]),
        list([-2.6149177556739924, 51.45400691005982]),
        list([-2.6148319249876697, 51.45400691005982])],
       [list([-2.63671875, 51.45406039482198]),
        list([-2.6366329193136773, 51.45406039482198]),
        list([-2.636547088627354, 51.45406039482198]), ...,
        list([-2.615003586360315, 51.45406039482198]),
        list([-2.6149177556739924, 51.45406039482198]),
        list([-2.6148319249876697, 51.45406039482198])],
       [list([-2.63671875, 51.45411387958414]),
        list([-2.6366329193136773, 51.45411387958414]),
        list([-2.636547088627354, 51.45411387958414]), ...,
        list([-2.615003586360315, 51.45411387958414]),
        list([-2.6149177556739924, 51.45411387958414]),
        list([-2.6148319249876697, 51.45411387958414])],
    

In [257]:
targetNode = lng_lat_to_coord(lng_lat_matrix, lng_lat=list(end_lng_lat))
targetNode
#print("startNode", startNode)
#print("targetNode", targetNode)
# Get elevation matrix

(69, 151)

In [258]:
elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)
elevation_mat

array([[109.1, 109.2, 109.3, ...,  76.4,  76.5,  76.6],
       [109.4, 109.5, 109.6, ...,  76.6,  76.7,  76.8],
       [109.8, 109.9, 110. , ...,  76.7,  76.8,  77. ],
       ...,
       [ 17.7,  17.8,  17.9, ...,   9.3,   9.4,   9.5],
       [ 17.3,  17.5,  17.6, ...,   9.2,   9.2,   9.3],
       [ 17.1,  17.3,  17.4, ...,   9. ,   9.1,   9.1]])

In [259]:
# Pad matrix with infinities to represent boundaries
padded_mat = np.pad(elevation_mat, [(1, 1), (1, 1)], mode='constant', constant_values=np.Inf)
padded_mat

array([[  inf,   inf,   inf, ...,   inf,   inf,   inf],
       [  inf, 109.1, 109.2, ...,  76.5,  76.6,   inf],
       [  inf, 109.4, 109.5, ...,  76.7,  76.8,   inf],
       ...,
       [  inf,  17.3,  17.5, ...,   9.2,   9.3,   inf],
       [  inf,  17.1,  17.3, ...,   9.1,   9.1,   inf],
       [  inf,   inf,   inf, ...,   inf,   inf,   inf]])

In [260]:
# resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
latitude_radians = tile_lng_lat[1] * math.pi / 180
resolution = abs(156543.03 * np.cos(latitude_radians) / (2 ** 14))

print("latitude_radians", latitude_radians)
print("resolution", resolution)

latitude_radians 0.8979598748325363
resolution 5.954508928963816


In [261]:
# Generate the shortest path as a sequence of lng, lat tuples
node_path = djikstra(padded_mat, startNode=startNode, targetNode=(targetNode[0]+1, targetNode[1]+1),
                        resolution=resolution, elevation_multiplier=5)
# node_path

In [262]:
# Get lng and lat of upper-left of tile
upper_left = mercantile.ul(tile_coords)
upper_left

LngLat(lng=-2.63671875, lat=51.45400691005982)

In [263]:
# Gets path as series of longitude and latitude coordinates
lnglatPath = [coord_to_lng_lat(upper_left, coord, 14) for coord in node_path]
lnglatPath
# if show_img:
#     plt.imshow(elevation_mat, interpolation='nearest')
#     xs = [x[0] for x in node_path]
#     ys = [x[1] for x in node_path]
#     plt.plot(xs, ys, 'r-')
#     plt.show()

minpath = convertToJson(lnglatPath)